# Multiclass support vector machine

In this exercise you are going to implement a multiclass support vector machine. You will implement:

* Load and preprocess the dataset.
* Implement loss function.
* Implement gradient descent.

In [ ]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import random
import sys
import time
sys.path.insert(0, "../tools/")
from utils import load_cifar10

In [ ]:
# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
    del X_train, y_train
    del X_test, y_test
    print('Clear previously loaded data.')
except:
    pass

X_train, y_train, X_test, y_test = load_cifar10()

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

## Data visualization

In [ ]:
"""
TODO: Visualize some examples from the dataset. Make a grid of images with matplotlib
      and try to make that each column belongs to a class
""" 

In [ ]:
# Split the data into train, val, and test sets. In addition we will
# create a small development set as a subset of the training data;
# we can use this for development so our code runs faster.
num_training = 5000
num_validation = 1000
num_test = 600
num_dev = 500

# Our validation set will be num_validation points from the original
# training set.
mask = range(num_training, num_training + num_validation)
X_val = X_train[mask]
y_val = y_train[mask]

# Our training set will be the first num_train points from the original
# training set.
mask = range(num_training)
X_train = X_train[mask]
y_train = y_train[mask]

# We will also make a development set, which is a small subset of
# the training set.
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]

# We use the first num_test points of the original test set as our
# test set.
mask = range(num_test)
X_test = X_test[mask]
y_test = y_test[mask]

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# Preprocessing: reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

# As a sanity check, print out the shapes of the data
print('Training data shape: ', X_train.shape)
print('Validation data shape: ', X_val.shape)
print('Test data shape: ', X_test.shape)
print('dev data shape: ', X_dev.shape)

In [ ]:
# Preprocessing: subtract the mean image
# Compute the image mean based on the training data
mean_image = np.mean(X_train, axis=0)

# print a few of the elements
print(mean_image[:10])

# Visualize the mean image using matplotlib                             
plt.figure(figsize=(4,4))
plt.imshow(mean_image.reshape((32,32,3)).astype('uint8'))
plt.show()

In [ ]:
# Subtract the mean to the training and testing set                     
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

In [ ]:
# third: append the bias dimension of ones (i.e. bias trick) so that our SVM
# only has to worry about optimizing a single weight matrix W.
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)

In [ ]:
def svm_loss_loop(W, X, y, reg, delta=1):
    """
    Structured SVM loss function, implementation with loops 

    D: number of features at the input vector
    C: number of classes in the dataset
    N: number of samples to operate on minibatches

    Inputs
     W: A numpy array of shape (D, C) containing weights.
     X: A numpy array of shape (N, D) containing a minibatch of data.
     y: A numpy array of shape (N,) containing training labels; y[i] = c means
    that X[i] has label c, where 0 <= c < C.
     reg: (float) regularization strength

    output:
     loss: the total loss as single float
     dW: gradients with respect to weights W; an array of same shape as W
    """
    dW = np.zeros(W.shape) # initialize the gradient as zero

    # compute the loss and the gradient
    num_classes = W.shape[1]
    num_train = X.shape[0]
    loss = 0.0
    for i in range(num_train):
        scores = X[i].dot(W)
        correct_class_score = scores[y[i]]
        diff_count = 0
        for j in range(num_classes):
            if j == y[i]:
                continue
            if margin > 0:
                loss = 0

    
    # TODO: compute loss and gradients for this function, take into account 
    # regularization in the loss and the gradients
    return loss, dW

In [ ]:
# generate a random SVM weight matrix of small numbers
W = np.random.randn(3073, 10) * 0.0001 

loss, grad = svm_loss_loop(W, X_dev, y_dev, 0)
print('loss: {}'.format(loss))

In [ ]:
# Numerically compute the gradient along several randomly chosen dimensions, and
# compare them with your analytically computed gradient. The numbers should match
# almost exactly along all dimensions.
loss, grad = svm_loss_loop(W, X_dev, y_dev, 0)

from gradient_check import grad_check_sparse
f = lambda w: svm_loss_loop(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad)

# do the gradient check once again with regularization turned on
# you didn't forget the regularization gradient did you?
loss, grad = svm_loss_loop(W, X_dev, y_dev, 5e1)
f = lambda w: svm_loss_loop(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad)

In [ ]:
def svm_loss_vectorization(W, X, y, reg, delta=1):
    """
    Structured SVM loss function, implementation with vectorization

    D: number of features at the input vector
    C: number of classes in the dataset
    N: number of samples to operate on minibatches

    Inputs
     W: A numpy array of shape (D, C) containing weights.
     X: A numpy array of shape (N, D) containing a minibatch of data.
     y: A numpy array of shape (N,) containing training labels; y[i] = c means
    that X[i] has label c, where 0 <= c < C.
     reg: (float) regularization strength

    Output:
     loss: the total loss as single float
     dW: gradients with respect to weights W; an array of same shape as W
    """
    loss = 0
    dW = np.zeros(W.shape)
    
    ##########################################################################
    # TODO: compute loss and gradients for this function, take into account  #
    # regularization in the loss and the gradients                           #
    # START                                                                  #
    ##########################################################################
    
    # compute loss
    
    ##########################################################################
    # END                                                                    #
    ##########################################################################
    return loss, dW

In [ ]:
# Complete the implementation of svm_loss_vectorization, and compute the gradient
# of the loss function in a vectorized way.

# The naive implementation and the vectorized implementation should match, but
# the vectorized version should still be much faster.
tic = time.time()
_, grad_naive = svm_loss_loop(W, X_train, y_train, 0.00001)
toc = time.time()
print('Naive loss and gradient: computed in {}'.format(toc - tic))

tic = time.time()
_, grad_vectorized = svm_loss_vectorization(W, X_train, y_train, 0.00001)
toc = time.time()
print('vectorized loss and gradient: computed in {}'.format(toc - tic))

# The loss is a single number, so it is easy to compare the values computed
# by the two implementations. The gradient on the other hand is a matrix, so
# we use the Frobenius norm to compare them.
difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('difference: {}'.format(difference))

In [ ]:
# TODO: Implement the svm classifier class using the loss previous function and plot the loss for the svm classifier
class customSVMClassifier(object):
    def __init__(self, X, Y, lr=1e-1, reg=1e-5, delta=1, num_iter=800, batch_size=100):
        self.x_train = X
        self.y_train = Y
        self.lr = lr
        self.reg = reg
        self.num_iter = num_iter
        self.batch_size = batch_size
        self.loss_history = list()
        self.delta = delta
        self.W = np.random.randn(X.shape[1], np.max(Y) + 1) * 0.01
    def svm_loss_vectorization(self, W, X, y, reg, delta):
        """
        Structured SVM loss function, implementation with vectorization

        D: number of features at the input vector
        C: number of classes in the dataset
        N: number of samples to operate on minibatches

        Inputs
         W: A numpy array of shape (D, C) containing weights.
         X: A numpy array of shape (N, D) containing a minibatch of data.
         y: A numpy array of shape (N,) containing training labels; y[i] = c means
        that X[i] has label c, where 0 <= c < C.
         reg: (float) regularization strength

        Output:
         loss: the total loss as single float
         dW: gradients with respect to weights W; an array of same shape as W
        """
        loss = 0
        dW = np.zeros(W.shape)

        ##########################################################################
        # TODO: compute loss and gradients for this function, take into account  #
        # regularization in the loss and the gradients                           #
        # START                                                                  #
        ##########################################################################

        
        ##########################################################################
        # END                                                                    #
        ##########################################################################
        return loss, dW
    def train(self):
        num_train = self.x_train.shape[0]
        for k in range(self.num_iter):
            X_batch = None
            y_batch = None

            #########################################################################
            # TODO:                                                                 #
            # Implement SGD (stochastic gradient descent)                           #
            # Hint: look for the function in numpy np.random.choice                 #
            #########################################################################
            # randomly sample some samples
            
            #########################################################################
            #  END                                                                  #               
            #########################################################################
    def predict(self, X):
        # TODO: Implement prediction function for svm
        #   X: input array with shape (N_test x D)
        pass
    def get_loss(self):
        return self.loss_history
    def getW(self):
        return self.W

In [ ]:
# Train a svm model and plot the loss

In [ ]:
# Write the LinearSVM.predict function and evaluate the performance on both the
# training and validation set
y_train_pred = svm.predict(X_train)
print('training accuracy: {}'.format(np.mean(y_train == y_train_pred)))
y_val_pred = svm.predict(X_val)
print('validation accuracy: {}'.format(np.mean(y_val == y_val_pred)))

In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of about 0.4 on the validation set.

learning_rates = [1e-7, 2e-7, 5e-7, 1e-6]
regularization_strengths = [1e4, 2e4, 5e4, 1e5, 5e5, 1e6]

# results is dictionary mapping tuples of the form
# (learning_rate, regularization_strength) to tuples of the form
# (training_accuracy, validation_accuracy). The accuracy is simply the fraction
# of data points that are correctly classified.
results = {}
best_val = -1   # The highest validation accuracy that we have seen so far.
best_svm = None # The LinearSVM object that achieved the highest validation rate.

################################################################################
# TODO:                                                                        #
# Write code that chooses the best hyperparameters by tuning on the validation #
# set. For each combination of hyperparameters, train a linear SVM on the      #
# training set, compute its accuracy on the training and validation sets, and  #
# store these numbers in the results dictionary. In addition, store the best   #
# validation accuracy in best_val and the LinearSVM object that achieves this  #
# accuracy in best_svm.                                                        #
#                                                                              #
# Hint: You should use a small value for num_iters as you develop your         #
# validation code so that the SVMs don't take much time to train; once you are #
# confident that your validation code works, you should rerun the validation   #
# code with a larger value for num_iters.                                      #
################################################################################

for learning in learning_rates:
    for regularization in regularization_strengths:
        pass

################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr {0} reg {1} train accuracy: {2} val accuracy: {3}'.format(lr, reg, train_accuracy, val_accuracy))
print('best validation accuracy achieved during cross-validation: {}'.format(best_val))

In [ ]:
# Visualize the learned weights for each class.
# Depending on your choice of learning rate and regularization strength, these may
# or may not be nice to look at.
w = best_svm.W[:-1,:] # strip out the bias
w = w.reshape(10, 32, 32, 3)
w_min, w_max = np.min(w), np.max(w)
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])